In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np

In [11]:
# Функция для создания самописной CNN без дообучения
def train_custom_cnn():
    class CustomCNN(nn.Module):
        def __init__(self):
            super(CustomCNN, self).__init__()
            self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
            self.relu1 = nn.ReLU()
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
            self.relu2 = nn.ReLU()
            self.fc = nn.Linear(32 * 8 * 8, 100)

        def forward(self, x):
            x = self.pool(self.relu1(self.conv1(x)))
            x = self.pool(self.relu2(self.conv2(x)))
            x = x.view(-1, 32 * 8 * 8)
            x = self.fc(x)
            return x

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка данных
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    # Создание модели и оптимизатора
    net = CustomCNN()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    # Обучение
    for epoch in range(20):
        running_loss = 0.0
        total_accuracy = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Расчет accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

        accuracy = 100 * total_accuracy / len(trainset)
        print(f'Эпоха {epoch + 1}, Потери: {running_loss / len(trainloader)}, Точность: {accuracy:.2f}%')

    print('Обучение самописной CNN завершено')

In [17]:
# Функция для создания CNN через дообучение ImageNet Resnet-50
def train_resnet50_transfer_learning():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка данных
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Изменение размера изображения для ResNet-50
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    # Загрузка предварительно обученной модели ResNet-50
    resnet = torchvision.models.resnet50(pretrained=True)
    num_ftrs = resnet.fc.in_features
    resnet.fc = nn.Linear(num_ftrs, 100)
    resnet.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

    # Обучение
    for epoch in range(5):
        running_loss = 0.0
        total_accuracy = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Расчет accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

        accuracy = 100 * total_accuracy / len(trainset)
        print(f'Эпоха {epoch + 1}, Потери: {running_loss / len(trainloader)}, Точность: {accuracy:.2f}%')

    print('Обучение CNN с использованием ImageNet ResNet-50 завершено')

In [18]:
# Функция для создания CNN через дообучение ImageNet Resnet-50 с аугментацией данных
def train_resnet50_with_augmentation():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Загрузка данных с аугментацией
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

    # Загрузка предварительно обученной модели ResNet-50
    resnet = torchvision.models.resnet50(pretrained=True)
    num_ftrs = resnet.fc.in_features
    resnet.fc = nn.Linear(num_ftrs, 100)
    resnet.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

    # Обучение
    for epoch in range(5):
        running_loss = 0.0
        total_accuracy = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Расчет accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_accuracy += (predicted == labels).sum().item()

        accuracy = 100 * total_accuracy / len(trainset)
        print(f'Эпоха {epoch + 1}, Потери: {running_loss / len(trainloader)}, Точность: {accuracy:.2f}%')

    print('Обучение CNN с использованием ImageNet ResNet-50 и аугментацией данных завершено')

In [15]:
# Обучение самописной CNN
train_custom_cnn()

Files already downloaded and verified
Эпоха 1, Потери: 4.588404572528342, Точность: 2.31%
Эпоха 2, Потери: 4.439851184025445, Точность: 4.22%
Эпоха 3, Потери: 4.115875048405679, Точность: 8.37%
Эпоха 4, Потери: 3.96037452361163, Точность: 10.52%
Эпоха 5, Потери: 3.866449200588724, Точность: 12.29%
Эпоха 6, Потери: 3.779471775454938, Точность: 13.46%
Эпоха 7, Потери: 3.711638938435508, Точность: 14.48%
Эпоха 8, Потери: 3.6617275400234917, Точность: 15.71%
Эпоха 9, Потери: 3.6164504024378785, Точность: 16.45%
Эпоха 10, Потери: 3.577941404279236, Точность: 17.25%
Эпоха 11, Потери: 3.526836884296154, Точность: 18.08%
Эпоха 12, Потери: 3.4808094153928635, Точность: 18.76%
Эпоха 13, Потери: 3.4326040494777357, Точность: 19.74%
Эпоха 14, Потери: 3.3858547320451273, Точность: 20.53%
Эпоха 15, Потери: 3.3510812414271753, Точность: 21.01%
Эпоха 16, Потери: 3.312873745818272, Точность: 21.76%
Эпоха 17, Потери: 3.2797129361525825, Точность: 22.42%
Эпоха 18, Потери: 3.247311655517734, Точность: 23.

In [19]:
# Обучение CNN с использованием ImageNet ResNet-50
train_resnet50_transfer_learning()

Files already downloaded and verified
Эпоха 1, Потери: 3.0779860233102005, Точность: 39.00%
Эпоха 2, Потери: 1.1647198255104787, Точность: 71.81%
Эпоха 3, Потери: 0.7616166585241743, Точность: 79.59%
Эпоха 4, Потери: 0.570047060394531, Точность: 84.21%
Эпоха 5, Потери: 0.43574659842664326, Точность: 88.01%
Обучение CNN с использованием ImageNet ResNet-50 завершено


In [20]:
# Обучение CNN с использованием ImageNet ResNet-50 и аугментацией данных
train_resnet50_with_augmentation()

Files already downloaded and verified
Эпоха 1, Потери: 4.007839771182946, Точность: 13.33%
Эпоха 2, Потери: 2.651599705066827, Точность: 37.34%
Эпоха 3, Потери: 2.0475451784670504, Точность: 48.37%
Эпоха 4, Потери: 1.7811646272459298, Точность: 53.61%
Эпоха 5, Потери: 1.6419980013766862, Точность: 56.56%
Обучение CNN с использованием ImageNet ResNet-50 и аугментацией данных завершено


На основе полученных результатов обучения трех различных моделей можно сделать следующие выводы:

1. Самописная CNN без дообучения:
   - Эпоха 1: Точность составляет всего 2.31%, что говорит о том, что модель пока не обучена и неспособна хорошо классифицировать изображения.
   - С каждой эпохой точность и потери постепенно улучшаются. На последних эпохах точность составляет около 24%, а потери уменьшаются до примерно 3.18.
   - В целом, результаты обучения самописной CNN не очень хорошие, и модель нуждается в дальнейшем улучшении.

2. CNN с использованием ImageNet ResNet-50 без аугментации данных:
   - Уже на первой эпохе точность составляет 39.00%, что гораздо лучше, чем у самописной CNN.
   - С каждой эпохой точность улучшается, достигая около 88.01% на последней эпохе. При этом потери уменьшаются до примерно 0.44.
   - Использование предварительно обученной ResNet-50 на ImageNet позволяет значительно улучшить результаты классификации на CIFAR-100 без аугментации данных, при этом возможен риск переобучения на тренировочных данных.

3. CNN с использованием ImageNet ResNet-50 и аугментацией данных:
   - На первой эпохе точность составляет всего 13.33%, что ниже, чем у ResNet-50 без аугментации.
   - Однако, с использованием аугментации данных модель показывает улучшение с каждой эпохой, достигая точности около 56.56% на последней эпохе.
   - Потери также уменьшаются до около 1.64 на последней эпохе.
   - Использование аугментации данных позволяет улучшить обучение модели, но точность все равно ниже, чем у ResNet-50 без аугментации.

Выводы:
- Использование предварительно обученной модели ResNet-50 на ImageNet существенно улучшает результаты классификации на CIFAR-100.
- Аугментация данных может помочь улучшить результаты, но в данном случае, несмотря на улучшение, точность остается ниже, чем у ResNet-50 без аугментации.
- В целом, использование предварительно обученных моделей и аугментации данных является полезной стратегией для улучшения результатов классификации изображений. Однако, возможно, еще можно оптимизировать гиперпараметры или провести более продолжительное обучение, чтобы получить еще более высокую точность модели.